In [ ]:
# Simplify and deploy on dot matrix at front
# check watchdog does or doesn't work
# report mdns not working on esp8266 (and the watchdog)
# check if timeout on wifi allows alternate wifi passwords to be given

# Make sure the firmware is
# %esptool esp8266 /home/julian/extrepositories/micropython/ports/esp8266/build-GENERIC/firmware-combined.bin

In [ ]:
# Small ESP8266

#      RST          |  GPIO1   TX
#       A0          |  GPIO3   RX
#       D0  GPIO16  |  GPIO5   D1  SCL
# SCK   D5  GPIO14  |  GPIO4   D2  SDA
# MISO  D6  GPIO12  |  GPIO0   D3
# MOSI  D7  GPIO13  |  GPIO2   D4  LED
# SS    D8  GPIO15  |          GND
#      3V3          |          5V

# clk red      D5
# cs  orange   D8
# din yellow   D7
# gnd green    G
# vcc blue     5V

In [155]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [156]:
%sendtofile config.txt

wifiname     BV6000
wifipassword bubblino
mqttbroker   192.168.43.1
pinled       2
pincs        15
pinsck       14
pinmosi      13
boardname    esp32ledstrip
numledchars  20
wifialt      DoESLiverpool,decafbad00,10.0.100.1

Sent 10 lines (222 bytes) to config.txt.


In [138]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.100.1
pinled       2
pincs        15
pinsck       14
pinmosi      13
boardname    esp8266led1
numledchars  20
wifialt      DoESLiverpool,decafbad00,10.0.100.1


Sent 10 lines (228 bytes) to config.txt.


In [157]:
%sendtofile ledstrippanel.py

import framebuf
from machine import Pin, SPI

cs, spi = None, None
brightnesschars = None
buffer = None
fbuff = None
numledchars = 0
# ESP32-HSPI: sck=14, mosi=13

def setbrightness(brightness, i0=0, i1=9999):
    # 0<=brightness<=15
    for i in range(max(i0, 0), min(i1, numledchars)):
        brightnesschars[(numledchars - i)*2 - 1] = brightness
        
def showbrightness():
    cs.value(0)  
    spi.write(brightnesschars)
    cs.value(1)
    
def scrollbrightness(brightness=0):
    for i in range(numledchars - 1):
        brightnesschars[(numledchars - i)*2 - 1] = brightnesschars[(numledchars - (i+1))*2 - 1]
    brightnesschars[1] = brightness
    
s = bytearray(8)
def show():
    for y in range(8):
        s[6] = s[4] = s[2] = s[0] = y+1
        yp = y*(numledchars) + numledchars
        cs.value(0)  # cannot put in loop as resets the shifting
        for m in range(0, numledchars-1, 4):
            s[1] = buffer[yp - m - 1]
            s[3] = buffer[yp - m - 2]
            s[5] = buffer[yp - m - 3]
            s[7] = buffer[yp - m - 4]
            spi.write(s)
        cs.value(1)

def init():
    for i in range(0, 10, 2):
        cs.value(0)
        x = b"\x0c\x00\x0f\x00\x0b\x07\x09\x00\x0c\x01"[i:i+2]
        spi.write(x*numledchars)
        cs.value(1)
    showbrightness()
        
def setup(lnumledchars, lcs, lspi):
    global numledchars, cs, spi, brightnesschars, buffer, fbuff
    cs, spi = lcs, lspi
    numledchars = lnumledchars # can be 4n+1 for scroll buffer

    brightnesschars = bytearray(bytes([0x0a, 0])*numledchars)
    buffer = bytearray(8*(numledchars))  # make one extra character to scroll from
    fbuffwidth = 8*(numledchars)
    fbuffheight = 8
    fbuff = framebuf.FrameBuffer(buffer, fbuffwidth, fbuffheight, framebuf.MONO_HLSB)

    init()
    
    for i in range(0, numledchars*8, 4):
        fbuff.fill_rect(i, (i%8), 4, 4, 1)
    show()
    return fbuff


Sent 64 lines (1913 bytes) to ledstrippanel.py.


In [153]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [158]:
%sendtofile main.py

from mqtt_as import config, MQTTClient
import uasyncio as asyncio
from machine import Pin, PWM, SPI
import time, itertools
from ledstrippanel import setup, show
from uasyncio.queues import Queue

fconfig = dict(x.split()  for x in open("config.txt"))
boardname = fconfig["boardname"]

pinled = Pin(int(fconfig["pinled"]), Pin.OUT)
pinled.value(0)
numledchars = int(fconfig["numledchars"])
fbuff = setup(numledchars, Pin(int(fconfig["pincs"]), Pin.OUT), SPI(1, 1000000))

time.sleep(2.5)

config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['mqttchannel'] = boardname
    
qmessages = Queue()
async def messagetask():
    message = " "
    while True:
        prevmessage, message = message, await(qmessages.get())
        for xchar, (c1, c2) in enumerate(zip(prevmessage, message)):
            if c1 != c2:
                break
        fbuff.fill(0)
        fbuff.text(message, 0, 0, 1)
        for ys in range(7, -1, -1):
            fbuff.fill_rect(xchar*8, 0, (numledchars - xchar)*8, 8, 0)
            fbuff.text(prevmessage[xchar:], xchar*8, ys - 8, 1)
            fbuff.text(message[xchar:], xchar*8, ys, 1)
            show()
            await asyncio.sleep_ms(50)
        await asyncio.sleep_ms(450)

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await qmessages.put("subscribing")
    await client.subscribe(boardname+"/message")
    await client.publish(boardname+'/ip', ipnumber, retain=True)
    await qmessages.put("ip: "+ipnumber)

async def mqtttask(bflip=False):
    if bflip and "wifialt" in fconfig:
        assid, awifi_pw, aserver = fconfig["wifialt"].split(",")
        if client._ssid == assid:
            assid, awifi_pw, aserver = fconfig["wifiname"], fconfig["wifipassword"], fconfig["mqttbroker"]
        client._ssid, client._wifi_pw, client.server = assid, awifi_pw, aserver
    await qmessages.put("wifi: "+client._ssid)
    try:
        await client.connect()
        await qmessages.put("mqtt: "+client.server)
        return
    except OSError as e:
        print("client connect", [e])
    await qmessages.put("wifi: fail")
    aloop.create_task(mqtttask(True))
    
async def cursorflash():    
    for i in itertools.count():
        await asyncio.sleep_ms(1000)
        if not client.isconnected() and qmessages.qsize() < 2:
            await qmessages.put("wifi fail: %d"%i)
            await asyncio.sleep_ms(9000)
        elif qmessages.empty():
            fbuff.fill_rect((numledchars-1)*8, 7, 8, 1, i%2)
            show()
        
def callbackcmd(topic, message, retain):
    print("callbackcmd", topic, message)
    while qmessages.qsize() > 5:
        qmessages.pop()
    if 1 <= len(message) <= 50:
        aloop.create_task(qmessages.put(message.decode()))
    
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
client = MQTTClient(config)

aloop = asyncio.get_event_loop()
aloop.create_task(messagetask())
aloop.create_task(cursorflash())
aloop.create_task(mqtttask())
aloop.run_forever()


Sent 90 lines (3105 bytes) to main.py.
